# 1. mô tả các thống kê tổng quan về tập dữ liệu (số mẫu, số đặc trưng của 1 mẫu, kiểu dữ liệu của mỗi đặc trưng, số mẫu dữ liệu trống của mỗi đặc trưng,...)

In [1]:
# import thu vien
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data = pd.read_csv("rawData.csv")
#data.drop(columns=['Unnamed: 12', 'Unnamed: 13'],inplace=True)


In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.isnull().sum()



In [ ]:
# data["area"] = data["area"].astype(float)
data["price"] = data["price"].replace("Thỏa thuận", "0")
data["price"] = data["price"].astype(float)
data.describe()

In [ ]:
# count = 0
# for i in range(len(data["price"])):
#     if data["price"][i] > 10 :
#         count += 1
#         # print(data1["price"][i])
#         data["price"] = data["price"].replace(data["price"][i], 0)
    
# # count 
# print(data)

# 2. xuất ra các thống kê mô tả trực quan về các đặc trưng (ví dụ: dùng boxplot, histogram, scatter plot về độ tương quan,…).


- Do tuong quan giua cac dac trung

In [ ]:
corr = data.corr().round(2)
mask = np.zeros_like(corr)
plt.subplots(figsize=(16, 8))
ax = sns.heatmap(corr, annot=True, linewidths=.5)

- phan bo cua dac trung

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 12))

fig.suptitle('Distribution Feature')

sns.histplot(data= data["area"], bins= 12, kde= False, ax= axes[0, 0])
sns.histplot(data= data["price"], bins= 12, kde= False, ax= axes[0, 1])
sns.histplot(data= data["number_bedroom"], bins= 12, kde= False, ax= axes[0, 2])
sns.histplot(data= data["number_wc"], bins= 12, kde= False, ax= axes[1, 0])
sns.histplot(data= data["number_floor"], bins= 12, kde= False, ax= axes[1, 1])
sns.histplot(data= data["entrance"], bins= 12, kde= False, ax= axes[1, 2])
sns.histplot(data= data["facade"], bins= 12, kde= False, ax= axes[2, 0])
sns.histplot(data= data["number_parking"], bins= 12, kde= False, ax= axes[2, 1])
sns.histplot(data= data["id_estate"], bins= 12, kde= False, ax= axes[2, 2])


In [ ]:
from typing import Tuple
from sklearn.base import BaseEstimator, TransformerMixin


def find_boxplot_boundaries(
    col: pd.Series, whisker_coeff: float = 1.5
) -> Tuple[float, float]:
    """Findx minimum and maximum in boxplot.

    Args:
        col: a pandas serires of input.
        whisker_coeff: whisker coefficient in box plot
    """
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - whisker_coeff * IQR
    upper = Q3 + whisker_coeff * IQR
    return lower, upper


class BoxplotOutlierClipper(BaseEstimator, TransformerMixin):
    def __init__(self, whisker_coeff: float = 1.5):
        self.whisker = whisker_coeff
        self.lower = None
        self.upper = None

    def fit(self, X: pd.Series):
        self.lower, self.upper = find_boxplot_boundaries(X, self.whisker)
        return self

    def transform(self, X):
        return X.clip(self.lower, self.upper)

In [ ]:


data["area"] = BoxplotOutlierClipper().fit_transform(data["area"])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
data["area"].hist(bins=50, ax=axes[0])
data["area"].to_frame().boxplot(ax=axes[1], vert=False)


In [ ]:
data["number_bedroom"] = BoxplotOutlierClipper().fit_transform(data["number_bedroom"])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
data["number_bedroom"].hist(bins=50, ax=axes[0])
data["number_bedroom"].to_frame().boxplot(ax=axes[1], vert=False);

In [ ]:
data["number_wc"] = BoxplotOutlierClipper().fit_transform(data["number_wc"])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
data["number_wc"].hist(bins=50, ax=axes[0])
data["number_wc"].to_frame().boxplot(ax=axes[1], vert=False);

In [ ]:
data["number_floor"] = BoxplotOutlierClipper().fit_transform(data["number_floor"])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
data["number_floor"].hist(bins=50, ax=axes[0])
data["number_floor"].to_frame().boxplot(ax=axes[1], vert=False);

- quan he giua gia ca voi cac dac trung

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

fig.suptitle('Distribution Feature')

sns.scatterplot(data=data, x="price", y="area", ax= axes[0, 0])
sns.scatterplot(data=data, x="price", y="number_bedroom", ax= axes[0, 1])
sns.scatterplot(data=data, x="price", y="number_wc", ax= axes[0, 2])
sns.scatterplot(data=data, x="price", y="number_floor", ax= axes[1, 0])


In [ ]:
# data["address"] = address_stat = data["address"].apply(lambda x: x.split(",")[len(x.split(",")) - 1])
data["address"] = data["address"].apply(lambda x: x.split(",")[len(x.split(",")) - 1])

address_stats = data.groupby('address')["address"].agg('count').sort_values(ascending= False)
address_stats

In [ ]:
data1 = data.copy()
data1['number_bedroom'] = data1['number_bedroom'].replace(np.nan, np.nanmedian(data1['number_bedroom']))
data1['number_wc'] = data1['number_wc'].replace(np.nan, np.nanmedian(data1['number_wc']))
data1['number_floor'] = data1['number_floor'].replace(np.nan, np.nanmedian(data1['number_floor']))
data1['entrance'] = data1['entrance'].replace(np.nan, np.nanmedian(data1['entrance']))
data1['facade'] = data1['facade'].replace(np.nan, np.nanmedian(data1['facade']))
data1['number_parking'] = data1['number_parking'].replace(np.nan, np.nanmedian(data1['number_parking']))
data1.isna().sum()
# print(np.nanmedian(data1['number_parking']))

In [ ]:
data1.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

gle = LabelEncoder()
genre_labels = gle.fit_transform(data1['address'])
genre_mappings = {index: label for index, label in 
                  enumerate(gle.classes_)}
# genre_mappings
data1


In [ ]:
# corr = data1.corr().round(2)
# mask = np.zeros_like(corr)
# plt.subplots(figsize=(16, 8))
# ax = sns.heatmap(corr, vmin=0, vmax=1, annot=True, linewidths=.5)

In [ ]:
# fig, axes = plt.subplots(3, 3, figsize=(20, 12))

# fig.suptitle('Distribution Feature')

# sns.displot(data= data1["area"])
# plt.show()
# sns.displot(data= data["price"], ax= axes[0, 1])
# sns.displot(data= data["number_bedroom"], ax= axes[0, 2])
# sns.displot(data= data["number_wc"], ax= axes[1, 0])
# sns.displot(data= data["number_floor"], ax= axes[1, 1])
# sns.displot(data= data["entrance"], ax= axes[1, 2])
# sns.displot(data= data["facade"], ax= axes[2, 0])
# sns.displot(data= data["number_parking"], ax= axes[2, 1])
# sns.displot(data= data["id_estate"], ax= axes[2, 2])
count = 0
for i in range(len(data1["price"])):
    if data["price"][i] > 10 :
        count += 1
        # print(data1["price"][i])
        data1["price"] = data1["price"].replace(data1["price"][i], 0)
    
# count 
print(data1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
from sklearn.linear_model import Ridge
from sklearn. impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector,make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score

In [ ]:
X=data1[["area", "number_bedroom", "number_wc", "id_estate", "address","number_floor"]]
y = data1["price"].astype(float)
X

In [ ]:
imp_median = SimpleImputer(strategy='median', add_indicator=True)
scaler = StandardScaler()
imp_constant = SimpleImputer(strategy='constant')
ohe = OneHotEncoder(handle_unknown='ignore')
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')
preprocessor = make_column_transformer(
    (make_pipeline(imp_median, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols)
)

In [ ]:

def pipeline_model1(X, y, data):
    pipe = make_pipeline(preprocessor, RandomForestRegressor())

    print(cross_val_score(pipe, X, y).mean())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state=32)

    pipe.fit(X_train, y_train)
    
    predict = pipe.predict(X_test)
    scores1 = cross_val_score( pipe, X, y, cv=5)
    print('Mean: {:.04f} {:.03f}'.format(np.mean(scores1), np.std(scores1)))
   
    print(f'MSE             : {metrics.mean_squared_error(y_test, predict)}\n'
          f'\nRMSE            : '
          f'{np.sqrt(metrics.mean_squared_error(y_test, predict))}\n'
          f'MAE             : {metrics.mean_absolute_error(y_test, predict)}\n'
          #f'Mean abs % error: '
         
          #f'{metrics.mean_absolute_percentage_error(y_test, predict)}\n'
          f'\n'
         f'Score (train)   : {pipe.score(X_train, y_train)}\n'
         f'Score (test)    : {pipe.score(X_test, y_test)}\n')
      
    comparison_of_results(X_test, y_test, pipe)
    visualize_model_results(data, pipe)
def pipeline_model2(X, y, data):
    pipe = make_pipeline(preprocessor, XGBRegressor())

    print(cross_val_score(pipe, X, y).mean())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state=32)

    pipe.fit(X_train, y_train)

    predict = pipe.predict(X_test)
    scores1 = cross_val_score( pipe, X, y, cv=5)
    print('Mean: {:.04f} {:.03f}'.format(np.mean(scores1), np.std(scores1)))
    
    print(f'MSE             : {metrics.mean_squared_error(y_test, predict)}\n'
          f'\nRMSE            : '
          f'{np.sqrt(metrics.mean_squared_error(y_test, predict))}\n'
          f'MAE             : {metrics.mean_absolute_error(y_test, predict)}\n'
          #f'Mean abs % error: '
          #f'{metrics.mean_absolute_percentage_error(y_test, predict)}\n'
          f'\n'
         
        f'Score (train)   : {pipe.score(X_train, y_train)}\n'
         f'Score (test)    : {pipe.score(X_test, y_test)}\n')
        
    comparison_of_results(X_test, y_test, pipe)
    visualize_model_results(data, pipe)

def comparison_of_results(X_test, y_test, model, times=5):
    for i in range(times):
        rnd = np.random.randint(0, y_test.shape[0] - 1)
        real = y_test.iloc[rnd]
        pred = float(model.predict(X_test.iloc[rnd].to_frame().T)[0])
        print(f'Real Value      ----->>>>> {real} $\n'
              f'Predicted Value ----->>>>> {pred} $')
        print()


def visualize_model_results(data, model):
    fig = plt.figure(figsize=(17, 10))
    data = data.sort_values(by=['price'])
    X = data[["area", "number_bedroom", "number_wc", "id_estate", "address","number_floor"]]
    y = data.price.astype(float)
    
    plt.scatter(range(X.shape[0]), y, color='red', label='Real')
    plt.scatter(range(X.shape[0]), model.predict(X), marker='.', label='Predict')

    plt.legend(loc=2, prop={'size': 25})

In [ ]:
pipeline_model1(X, y, data1)

In [ ]:
pipeline_model2(X, y, data1)